## Algo de reconstruction des pales d'éoliennes

retrouver la bonne surface

In [1]:
import os
import sys
import math
from math import *
import time
import numpy as np
from scipy.interpolate import splprep, splev
from airfoil_mesh import * 
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:




def construct_init(airfoil_list, profiles_cut = [0], profiles_chord_lenght = [], rot= 0, L= 40, alpha = 0, sweep_chord=0, ratio = 0.363, lc=0.005, output_name = 'output', i = 0, fuite = [1.5877, 0, 2.7345]):
    
    ################################## How to complete the code
    ###MANDATORY
    #profiles: list of filename where we reed Values.txt  >>> les différents profils en nom de DOSSIER

    ###MAIN OPTIONAL
    #profiles_cut: list of float from 0 to 100 corresponding to the distance to the basis of each profile (in % L the total lenght of the airfoil). Warning: must be the same lenght as profiles
    #profiles_chord_lenght: list of the chord lenght of each profile. Warning must be the same lenght as profiles
    #L: float corresponding to the lenght of the wing from basis to tip
    #rot: list (in degrees) of angle of rotation of tip chord vs basis chord. Usefull for wind turbines
    
    #alpha, sweep_chord, ratio: (in degrees) angle of sweep, float from 0 to 1 corresponding to the point of the chord (in %) having a linear translation of angle alpha from axis X, ratio of wing (tip lenght / basis lenght)
    #fuite: list of lenght=3, center of homothetical transformation of the wing (king of conical extrudation). Can be determined by (L, alpha, sweep_chord, ratio)
    #i: in degrees, general incidence of the wing post creation. Is not the incidence of each profil which can be selected with rot
    
    #MINOR OPTIONAL
    #lc: mesh size parameter (recommended < 0.01)
    #output_name: 'output' gives a filenam 'naca_output.geo'


    ##########################################################################Préliminaires
    
    
    texte =[]
    texte.append("""SetFactory("OpenCASCADE"); \n \n""")

    ###########################################################################Données

    if (L, alpha, sweep_chord, ratio) != (1.1963, 0, 0, 0.56):
        fuite[2]=L/(1- ratio)
        fuite[0]= fuite[2]*tan(alpha) + sweep_chord

    m=[]
    if profiles_chord_lenght != []:
        for j in range(len(profiles_chord_lenght)):
            m.append(profiles_chord_lenght[j])
    else:
        for j in range(len(profiles_cut)):
            m.append(1-(profiles_cut[j]*(L/fuite[2])/100))
    
    texte.append("""///////////////////////////////////////////////////////////Données \n \n""")
    texte.append(f"L = " + f"{float(L)}" + "; \n")
    texte.append(f"alpha = " + f"{float(alpha)}" + "; \n")
    texte.append(f"K = " + f"{float(ratio)}" + "; \n")
    texte.append(f"X = " + f"{float(fuite[0])}" + "; \n")
    texte.append(f"Y = " + f"{float(fuite[1])}" + "; \n")
    texte.append(f"Z = " + f"{float(fuite[2])}" + "; \n")
    texte.append(f"i = " + f"{float(i)}" + "; \n")
    texte.append(f"lc = " + f"{float(lc)}" + "; \n")
    #♣texte.append(f"rot= {rot}*Pi/180; \n")                               
    texte.append("""///////////////////////////////////////////////////////////FIN des Données \n \n""")



    nb_pts=145                                                                        #a changer


    nb_pro = len(airfoil_list)
    for j in range(nb_pro):
        
        curve = airfoil_list[j]

        texte.append("///////////////////////////////////////////////////////////Profil n° " + f"{j}" + "\n \n")
        for k in range(len(curve)):
            texte.append(f"Point({k+1+j*200}) = " + "{"+ f"{float(curve[k][0])}, {float(curve[k][1])}, 0, lc" +"};\n")
        texte.append("////////On relie avec des B-splines; \n")
        texte.append(f"BSpline({1+10*j}) =" + "{" + f"{j*200 + 1} : {j*200 + nb_pts} " + "}; \n")
        #texte.append(f"BSpline({2+10*j}) =" + "{" + f"{j*200 + 61} : {j*200 + 89} " + "}; \n")
        #texte.append(f"BSpline({3+10*j}) =" + "{" + f"{j*200 + 89} : {j*200 + 145} " + "}; \n")
        texte.append(f"Line({2+10*j}) =" + "{" + f"{j*200 + nb_pts} , {j*200 + 1} " + "}; \n")
        
        texte.append(f"m = {m[j]}; \n")
        texte.append(f"c = 1-(({profiles_cut[j]}/100)*(L/Z)); \n")
        texte.append(f"z= {profiles_cut[j]}*L/100; \n")
        texte.append(f"x= K + z*{tan(alpha)}; \n")
        texte.append(f"y = 0; \n")
        texte.append("Translate { x - K, y, z} { \n")
        texte.append("  Curve"+"{"+ f"{1+10*j}"  +"}; "  + "Curve"+"{"+ f"{2+10*j}"+"}; \n") #+"Curve"+"{"+ f"{3+10*j}"+"}; "+"Curve"+"{"+ f"{4+10*j}"
        texte.append("} \n")
        #texte.append("Dilate" + " {"+"{ x, y, z}, {m, m, 0}} { \n")
        #texte.append("  Curve"+"{"+ f"{1+10*j}"  +"}; "  + "Curve"+"{"+ f"{2+10*j}"+"}; \n") #+"Curve"+"{"+ f"{3+10*j}"+"}; "+"Curve"+"{"+ f"{4+10*j}"
        #texte.append("} \n")
        texte.append("Curve Loop"+"("+ f"{1+j}"+") = {" + f"{10*j+1},"+ f"{2+10*j}" + "}; \n")
        texte.append("Plane Surface"+"("+ f"{1+j}"+") = {" + f"{1+j}"+"}; \n")
        #texte.append("Rotate" + " {"+"{0, 0, 1}, { x, y, z}, " + f"{rot[j]}" + "*Pi/180} { \n")
        #texte.append("  Surface"+"{" + f"{1+j}"+"}; \n")
        #texte.append("} \n")
    

    texte.append("\n\n\n")
    texte.append("///////////////////////////////////////////////////////////On les relie \n")
    for m in range(1,len(airfoil_list)):
        print('m',m)
        texte.append("ThruSections("+f"{m}"") = {"+f"{m}:"+ f"{m +1}" + "}; \n\n")
    #texte.append("ThruSections(1) = {1:" + f"{nb_pro }" + "}; \n\n")
    #texte.append("Rotate "+"{"+"{0, 0, 1}, {0.25, 0.03, 0}, -i*Pi/180} { \n")
    #texte.append("  Volume"+"{"+"1}; Surface"+"{"+"2}; Surface"+"{"+"6}; Surface"+"{"+"3}; Surface"+"{"+"4}; Surface"+"{"+"1}; Surface"+"{"+"5}; Surface"+"{"+"7}; \n")
    #texte.append("}")

    
    name_file = f"pale_{output_name}.geo"

    with open(name_file, 'w') as f:
        for element in texte:
            f.write(element)
    #mesh= str('gmsh -2 -format msh3 ' + f"naca_{output_name}.geo -o " + f"naca_{output_name}.msh" )
    #os.system(mesh)


In [3]:
## On trouve les coordonnées de la blade initiale

naca_63_215 = np.array([ 0.02267468,  0.03502696,  0.08531651,  0.08768866,  0.05825892,
       -0.05514708, -0.06891857, -0.0253396 ,  0.74175824, -0.00467257])

naca_63_415 = np.array([ 0.02312831,  0.04979696,  0.09752639,  0.09891683,  0.06405409,
       -0.04646897, -0.05629325, -0.01144421,  0.72674268, -0.00904017])
rad_to_deg = np.pi/180

naca_0 = airfoil(path='msh', geo=naca_63_215, corde=0.192, index=0, angle=0)
naca_0.write_mesh('0')

naca_1 = airfoil(path='msh', geo=naca_63_215, corde=0.172, index=1, angle=-2.5*rad_to_deg)
naca_1.write_mesh('1')

naca_2 = airfoil(path='msh', geo=naca_63_215, corde=0.151, index=2, angle=-5*rad_to_deg)
naca_2.write_mesh('2')

naca_3 = airfoil(path='msh', geo=naca_63_215, corde=0.131, index=3, angle=-7.5*rad_to_deg)
naca_3.write_mesh('3')

naca_4 = airfoil(path='msh', geo=naca_63_415, corde=0.111, index=4, angle=-10*rad_to_deg)
naca_4.write_mesh('4')

naca_5 = airfoil(path='msh', geo=naca_63_415, corde=0.091, index=5, angle=-12.5*rad_to_deg)
naca_5.write_mesh('5')

naca_6 = airfoil(path='msh', geo=naca_63_415, corde=0.07, index=6, angle=-15*rad_to_deg)
naca_6.write_mesh('6')



curve_0 = naca_0.curve_generation(naca_0.x_0)
curve_1 = naca_1.curve_generation(naca_0.x_0)
curve_2 = naca_2.curve_generation(naca_0.x_0)
curve_3 = naca_3.curve_generation(naca_0.x_0)
curve_4 = naca_4.curve_generation(naca_0.x_0)
curve_5 = naca_5.curve_generation(naca_0.x_0)
curve_6 = naca_6.curve_generation(naca_0.x_0)



list_curve_init = [curve_0, curve_1, curve_2, curve_3, curve_4, curve_5, curve_6]

contour[1, 2, 3, 4, 5]
Info    : Starting subloop 1 in curve loop 1 (are you sure about this?)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Nurb)
Info    : Done meshing 1D (Wall 0.0713494s, CPU 0.04818s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 2.63773s, CPU 2.53437s)
Info    : 20437 nodes 40735 elements
Info    : Writing 'msh/maillage0.msh'...
Info    : Done writing 'msh/maillage0.msh'
done
contour[1, 2, 3, 4, 5]
Info    : Starting subloop 1 in curve loop 1 (are you sure about this?)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Nurb)
Info    : Done meshing 1D (Wall 0.0420582s, 

In [4]:
## On construit la blade initiale

construct_init(list_curve_init, profiles_cut = [0,47,64,73,78,81,84], rot= np.zeros(len(list_curve_init)), L= 1.6, alpha = 0, sweep_chord=0, ratio = 0.07, lc=0.005, output_name = 'initiale')

m 1
m 2
m 3
m 4
m 5
m 6


## Construction de la pale finale 

In [5]:
face_0 = np.array([ 0.0289   ,  0.0004025,  0.09313  ,  0.08978  ,  0.1188   ,
        0.       , -0.09985  , -0.01279  ,  0.3742   , -0.09708  ])
face_1 =np.array([ 0.01211 ,  0.04379 ,  0.08971 ,  0.06649 ,  0.08487 , -0.001573,
       -0.003513, -0.1     ,  0.7963  , -0.0997  ])
face_2 =np.array([ 0.006597,  0.04929 ,  0.03788 ,  0.09194 ,  0.06355 , -0.03934 ,
       -0.0724  , -0.01171 ,  0.7536  ,  0.2995  ])
face_3 = np.array([ 0.01172 ,  0.06209 ,  0.08116 ,  0.09348 ,  0.1211  , -0.00779 ,
       -0.02546 , -0.004806,  0.7659  ,  0.2979  ])
face_4 = np.array([ 5.000e-03,  6.783e-02,  8.308e-02,  1.040e-01,  1.158e-01,
       -4.951e-03, -2.133e-02, -5.754e-04,  7.669e-01,  2.995e-01])
face_5 = np.array([ 1.564e-02,  5.553e-02,  9.739e-02,  1.188e-01,  1.276e-01,
       -5.683e-03, -5.469e-04, -3.310e-03,  7.835e-01,  2.996e-01])
face_6 = np.array([ 0.005238,  0.07    ,  0.1195  ,  0.05664 ,  0.1294  , -0.007883,
       -0.02563 , -0.006191,  0.6098  ,  0.299   ])




naca_0 = airfoil(path='msh', geo=face_0, corde=0.192, index=0, angle=0)
naca_0.write_mesh('0')

naca_1 = airfoil(path='msh', geo=face_1, corde=0.172, index=1, angle=-2.5*rad_to_deg)
naca_1.write_mesh('1')

naca_2 = airfoil(path='msh', geo=face_2, corde=0.151, index=2, angle=-5*rad_to_deg)
naca_2.write_mesh('2')

naca_3 = airfoil(path='msh', geo=face_3, corde=0.131, index=3, angle=-7.5*rad_to_deg)
naca_3.write_mesh('3')

naca_4 = airfoil(path='msh', geo=face_4, corde=0.111, index=4, angle=-10*rad_to_deg)
naca_4.write_mesh('4')

naca_5 = airfoil(path='msh', geo=face_5, corde=0.091, index=5, angle=-12.5*rad_to_deg)
naca_5.write_mesh('5')

naca_6 = airfoil(path='msh', geo=face_6, corde=0.07, index=6, angle=-15*rad_to_deg)
naca_6.write_mesh('6')



curve_0 = naca_0.curve_generation(naca_0.x_0)
curve_1 = naca_1.curve_generation(naca_1.x_0)
curve_2 = naca_2.curve_generation(naca_2.x_0)
curve_3 = naca_3.curve_generation(naca_3.x_0)
curve_4 = naca_4.curve_generation(naca_4.x_0)
curve_5 = naca_5.curve_generation(naca_5.x_0)
curve_6 = naca_6.curve_generation(naca_6.x_0)


list_curve_final = [curve_0, curve_1, curve_2, curve_3, curve_4, curve_5, curve_6]

contour[1, 2, 3, 4, 5]
Info    : Starting subloop 1 in curve loop 1 (are you sure about this?)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Nurb)
Info    : Done meshing 1D (Wall 0.0629909s, CPU 0.057315s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 2.92955s, CPU 2.7916s)
Info    : 22088 nodes 44037 elements
Info    : Writing 'msh/maillage0.msh'...
Info    : Done writing 'msh/maillage0.msh'
done
contour[1, 2, 3, 4, 5]
Info    : Starting subloop 1 in curve loop 1 (are you sure about this?)
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 20%] Meshing curve 2 (Line)
Info    : [ 40%] Meshing curve 3 (Line)
Info    : [ 60%] Meshing curve 4 (Line)
Info    : [ 80%] Meshing curve 5 (Nurb)
Info    : Done meshing 1D (Wall 0.0396563s, 

In [6]:
## On construit la blade finale

construct_init(list_curve_final, profiles_cut = [0,47,64,73,78,81,84], rot= np.zeros(len(list_curve_init)), L= 1.6, alpha = 0, sweep_chord=0, ratio = 0.07, lc=0.005, output_name = 'finale')

m 1
m 2
m 3
m 4
m 5
m 6


In [7]:
naca_3.x_0

array([ 0.01172 ,  0.06209 ,  0.08116 ,  0.09348 ,  0.1211  , -0.00779 ,
       -0.02546 , -0.004806,  0.7659  ,  0.2979  ])